In [ ]:
!pip install dash

In [ ]:
!pip install jupyter_dash

In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
JupyterDash.infer_jupyter_proxy_config()

In [ ]:
# Read the spacex data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()


# Create a dash application
app = dash.Dash(__name__)
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site_dropdown',
                                             options=[{'label': 'All Sites', 'value': 'All Sites'},
                                                      {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                      {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                      {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                      {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}],
                                             multi=False, placeholder='Select a Launch Site here',searchable=True),
                                html.Div(id='table-container'),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                #html.P("Graph:"),
                                html.Div(dcc.Graph(id='success-pie-chart',figure='figure')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload_slider',min=0,max=10000,step=1000,value=[min_payload,max_payload],
                                               marks={
                                                   0: {'label': '0', 'style': {'color': 'black'}},
                                                   2500: {'label': '2500', 'style': {'color': 'black'}},
                                                   5000: {'label': '5000', 'style': {'color': 'black'}},
                                                   10000: {'label': '10000', 'style': {'color': 'black'}}
                                               },updatemode='drag'),
                               # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart'))])

@app.callback( Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site_dropdown', component_property='value'))
def site(site_dropdown):
    if site_dropdown in ['CCAFS LC-40','VAFB SLC-4E','KSC LC-39A','CCAFS SLC-40']:
        df =  spacex_df[spacex_df['Launch Site']==str(site_dropdown)]
        ttle=str(site_dropdown)
        fig=px.pie(df,values=df.groupby(['class'])['Launch Site'].count(),names=df['class'].unique(),
                  title='Total Success Launches by Site '+ttle)
    if site_dropdown == 'All Sites':
        df =  spacex_df[['Launch Site','class']]
        fig=px.pie(df,values=df['class'],names=df['Launch Site'],title='Total Success Launches by Sites')
    else:
        print('Please select from the dropdown menu')
    return fig

@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              #Input(component_id="payload_slider", component_property="value"),
             Input(component_id='site_dropdown', component_property='value'))
def slider(site_dropdown):
    if site_dropdown=='All Sites':
        fig=px.scatter(spacex_df,x=spacex_df['Payload Mass (kg)'], y=spacex_df['class'],color="Booster Version Category",
                       title='Correlation between Payload and Success for all Sites')
    if site_dropdown in ['CCAFS LC-40','VAFB SLC-4E','KSC LC-39A','CCAFS SLC-40']: 
        df=spacex_df[spacex_df['Launch Site']==str(site_dropdown)]
        ttle=str(site_dropdown)
        fig=px.scatter(df,x=df['Payload Mass (kg)'], y=df['class'],color="Booster Version Category",
                       title='Correlation between Payload and Success for Site '+ttle)
    else:
        print('Please select from the available options')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(host="localhost", port=8050,debug=False, dev_tools_ui=False, dev_tools_props_check=False)